##Machine learning with Spark
Build your first Machine learning program.
You will proceed as follow:
* Step 1) Basic operation with PySpark
* Step 2) Data preprocessing
* Step 3) Build a data processing pipeline
* Step 4) Build the classifier
* Step 5) Train and evaluate the model
* Step 6) Tune the hyperparameter

## **Step 1) Basic operation with PySpark**

In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=39960a5249a16d8a56596a6039482c737ab46dcb607d2232b4cf488e75cda4b3
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [ ]:
import pyspark
from pyspark import SparkContext
sc =SparkContext()

In [ ]:
from pyspark import SparkFiles
url = "https://raw.githubusercontent.com/guru99-edu/R-Programming/master/adult_data.csv"
sc.addFile(url)

In [ ]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [ ]:
#Todo: using SQLContext to read csv and assign to dataframe
df = sqlContext.read.csv("adult_data.csv", header=True, inferSchema= True)

In [ ]:
#Todo:printSchema
df.printSchema()

root
 |-- x: integer (nullable = true)
 |-- age: integer (nullable = true)
 |-- workclass: string (nullable = true)
 |-- fnlwgt: integer (nullable = true)
 |-- education: string (nullable = true)
 |-- educational-num: integer (nullable = true)
 |-- marital-status: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- relationship: string (nullable = true)
 |-- race: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- capital-gain: integer (nullable = true)
 |-- capital-loss: integer (nullable = true)
 |-- hours-per-week: integer (nullable = true)
 |-- native-country: string (nullable = true)
 |-- income: string (nullable = true)



In [ ]:
df.show(5, truncate = False)

+---+---+---------+------+------------+---------------+------------------+-----------------+------------+-----+------+------------+------------+--------------+--------------+------+
|x  |age|workclass|fnlwgt|education   |educational-num|marital-status    |occupation       |relationship|race |gender|capital-gain|capital-loss|hours-per-week|native-country|income|
+---+---+---------+------+------------+---------------+------------------+-----------------+------------+-----+------+------------+------------+--------------+--------------+------+
|1  |25 |Private  |226802|11th        |7              |Never-married     |Machine-op-inspct|Own-child   |Black|Male  |0           |0           |40            |United-States |<=50K |
|2  |38 |Private  |89814 |HS-grad     |9              |Married-civ-spouse|Farming-fishing  |Husband     |White|Male  |0           |0           |50            |United-States |<=50K |
|3  |28 |Local-gov|336951|Assoc-acdm  |12             |Married-civ-spouse|Protective-serv 

In [ ]:
#rename columns to match code from PDF
cols = ['x', 'age','workclass','fnlwgt','education','education_num','marital_status','occupation','relationship','race','gender',
        'capital_gain','capital_loss','hours_week','native_country','label']

#note income -renamed-> as label
df=df.toDF(*cols)

In [ ]:
# Import all from `sql.types`
from pyspark.sql.types import *

# Write a custom function to convert the data type of DataFrame columns
def convertColumn(df, names, newType):
    for name in names:
        df = df.withColumn(name, df[name].cast(newType))
    return df
# List of continuous features
CONTI_FEATURES  = ['age', 'fnlwgt','capital_gain', 'education_num', 'capital_loss', 'hours_week']
# Convert the type
df = convertColumn(df, CONTI_FEATURES, FloatType())
# Check the dataset
df.printSchema()

root
 |-- x: integer (nullable = true)
 |-- age: float (nullable = true)
 |-- workclass: string (nullable = true)
 |-- fnlwgt: float (nullable = true)
 |-- education: string (nullable = true)
 |-- education_num: float (nullable = true)
 |-- marital_status: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- relationship: string (nullable = true)
 |-- race: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- capital_gain: float (nullable = true)
 |-- capital_loss: float (nullable = true)
 |-- hours_week: float (nullable = true)
 |-- native_country: string (nullable = true)
 |-- label: string (nullable = true)



In [ ]:
#Select Columns
df.select('age', 'fnlwgt').show(5)

+----+--------+
| age|  fnlwgt|
+----+--------+
|25.0|226802.0|
|38.0| 89814.0|
|28.0|336951.0|
|44.0|160323.0|
|18.0|103497.0|
+----+--------+
only showing top 5 rows



In [ ]:
#Count by Group
df.groupBy("education").count().sort("count",ascending=True).show()

+------------+-----+
|   education|count|
+------------+-----+
|   Preschool|   83|
|     1st-4th|  247|
|     5th-6th|  509|
|   Doctorate|  594|
|        12th|  657|
|         9th|  756|
| Prof-school|  834|
|     7th-8th|  955|
|        10th| 1389|
|  Assoc-acdm| 1601|
|        11th| 1812|
|   Assoc-voc| 2061|
|     Masters| 2657|
|   Bachelors| 8025|
|Some-college|10878|
|     HS-grad|15784|
+------------+-----+



In [ ]:
#describe data
df.describe().show()

+-------+------------------+------------------+-----------+------------------+------------+------------------+--------------+----------------+------------+------------------+------+------------------+-----------------+------------------+--------------+-----+
|summary|                 x|               age|  workclass|            fnlwgt|   education|     education_num|marital_status|      occupation|relationship|              race|gender|      capital_gain|     capital_loss|        hours_week|native_country|label|
+-------+------------------+------------------+-----------+------------------+------------+------------------+--------------+----------------+------------+------------------+------+------------------+-----------------+------------------+--------------+-----+
|  count|             48842|             48842|      48842|             48842|       48842|             48842|         48842|           48842|       48842|             48842| 48842|             48842|            48842|     

In [ ]:
df.describe('capital_gain').show()

+-------+------------------+
|summary|      capital_gain|
+-------+------------------+
|  count|             48842|
|   mean|1079.0676262233324|
| stddev| 7452.019057655413|
|    min|               0.0|
|    max|           99999.0|
+-------+------------------+



In [ ]:
#Crossab computation
df.crosstab('age', 'label').sort("age_label").show()

+---------+-----+----+
|age_label|<=50K|>50K|
+---------+-----+----+
|     17.0|  595|   0|
|     18.0|  862|   0|
|     19.0| 1050|   3|
|     20.0| 1112|   1|
|     21.0| 1090|   6|
|     22.0| 1161|  17|
|     23.0| 1307|  22|
|     24.0| 1162|  44|
|     25.0| 1119|  76|
|     26.0| 1068|  85|
|     27.0| 1117| 115|
|     28.0| 1101| 179|
|     29.0| 1025| 198|
|     30.0| 1031| 247|
|     31.0| 1050| 275|
|     32.0|  957| 296|
|     33.0| 1045| 290|
|     34.0|  949| 354|
|     35.0|  997| 340|
|     36.0|  948| 400|
+---------+-----+----+
only showing top 20 rows



In [ ]:
#Drop column
df.drop('education_num').columns

['x',
 'age',
 'workclass',
 'fnlwgt',
 'education',
 'marital_status',
 'occupation',
 'relationship',
 'race',
 'gender',
 'capital_gain',
 'capital_loss',
 'hours_week',
 'native_country',
 'label']

In [ ]:
#Filter Data
df.dropna()
df.filter(df.age > 40).count()

20211

In [ ]:
#Descriptive satisics by group
df.groupby('marital_status').agg({'capital_gain': 'mean'}).show()

+--------------------+------------------+
|      marital_status| avg(capital_gain)|
+--------------------+------------------+
|           Separated| 581.8424836601307|
|       Never-married|  384.382639449029|
|Married-spouse-ab...| 629.0047770700637|
|            Divorced| 793.6755615860094|
|             Widowed| 603.6442687747035|
|   Married-AF-spouse|2971.6216216216217|
|  Married-civ-spouse|1739.7006121810625|
+--------------------+------------------+



## Step 2) Data preprocessing

In [ ]:
# Add age square

from pyspark.sql.functions import *

# 1 Select the column
age_square = df.select(col("age")**2)

# 2 Apply the transformation and add it to the DataFrame
df = df.withColumn("age_square", col("age")**2)

df.printSchema()

root
 |-- x: integer (nullable = true)
 |-- age: float (nullable = true)
 |-- workclass: string (nullable = true)
 |-- fnlwgt: float (nullable = true)
 |-- education: string (nullable = true)
 |-- education_num: float (nullable = true)
 |-- marital_status: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- relationship: string (nullable = true)
 |-- race: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- capital_gain: float (nullable = true)
 |-- capital_loss: float (nullable = true)
 |-- hours_week: float (nullable = true)
 |-- native_country: string (nullable = true)
 |-- label: string (nullable = true)
 |-- age_square: double (nullable = true)



In [ ]:
COLUMNS = ['x', 'age', 'age_square', 'workclass','fnlwgt','education','education_num','marital_status','occupation','relationship','race','gender',
        'capital_gain','capital_loss','hours_week','native_country','label']

df = df.select(COLUMNS)
df.first()

Row(x=1, age=25.0, age_square=625.0, workclass='Private', fnlwgt=226802.0, education='11th', education_num=7.0, marital_status='Never-married', occupation='Machine-op-inspct', relationship='Own-child', race='Black', gender='Male', capital_gain=0.0, capital_loss=0.0, hours_week=40.0, native_country='United-States', label='<=50K')

In [ ]:
#Exclude Holand-Netherlands
df.filter(df.native_country == 'Holand-Netherlands').count()
df.groupby('native_country').agg({'native_country': 'count'}).sort(asc("count(native_country)")).show()

+--------------------+---------------------+
|      native_country|count(native_country)|
+--------------------+---------------------+
|  Holand-Netherlands|                    1|
|             Hungary|                   19|
|            Honduras|                   20|
|            Scotland|                   21|
|                Laos|                   23|
|          Yugoslavia|                   23|
|Outlying-US(Guam-...|                   23|
|     Trinadad&Tobago|                   27|
|            Cambodia|                   28|
|                Hong|                   30|
|            Thailand|                   30|
|             Ireland|                   37|
|              France|                   38|
|             Ecuador|                   45|
|                Peru|                   46|
|              Greece|                   49|
|           Nicaragua|                   49|
|                Iran|                   59|
|              Taiwan|                   65|
|         

In [ ]:
df_remove = df.filter(df.native_country!='Holand-Netherlands')

In [ ]:
df_remove.groupby('native_country').agg({'native_country': 'count'}).sort(asc("count(native_country)")).show()

+--------------------+---------------------+
|      native_country|count(native_country)|
+--------------------+---------------------+
|             Hungary|                   19|
|            Honduras|                   20|
|            Scotland|                   21|
|                Laos|                   23|
|          Yugoslavia|                   23|
|Outlying-US(Guam-...|                   23|
|     Trinadad&Tobago|                   27|
|            Cambodia|                   28|
|                Hong|                   30|
|            Thailand|                   30|
|             Ireland|                   37|
|              France|                   38|
|             Ecuador|                   45|
|                Peru|                   46|
|              Greece|                   49|
|           Nicaragua|                   49|
|                Iran|                   59|
|              Taiwan|                   65|
|            Portugal|                   67|
|         

## Step 3) Build a data processing pipeline


In [ ]:
#import libraries for pipeline
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder#Estimator
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler

In [ ]:
# 1. Encode the categorical data
CATE_FEATURES = ['workclass', 'education', 'marital_status', 'occupation', 'relationship', 'race', 'gender', 'native_country']

# stages in our Pipeline
stages = []


for categoricalCol in CATE_FEATURES:
    stringIndexer = StringIndexer(inputCol=categoricalCol, outputCol=categoricalCol + "Index")

    encoder = OneHotEncoder(inputCols=[stringIndexer.getOutputCol()],
                                     outputCols=[categoricalCol + "classVec"])

    stages += [stringIndexer, encoder]

In [ ]:
# 2. Index the label feature
# Convert label into label indices using the StringIndexer
label_stringIdx =  StringIndexer(inputCol="label", outputCol="newlabel")
stages += [label_stringIdx]

In [ ]:
# 3. Add continuous variable
assemblerInputs = [c + "classVec" for c in CATE_FEATURES] + CONTI_FEATURES

In [ ]:
# 4. Assemble the steps
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
stages += [assembler]

In [ ]:
stages

[StringIndexer_f443957294b0,
 OneHotEncoder_8d25f5d92ab6,
 StringIndexer_be0a44948255,
 OneHotEncoder_07b5db9fc386,
 StringIndexer_620018d361b9,
 OneHotEncoder_db8f90fbba47,
 StringIndexer_de2d683e28f7,
 OneHotEncoder_384a4a96bc47,
 StringIndexer_d81e4d94f940,
 OneHotEncoder_7e728582b638,
 StringIndexer_38e8cf742431,
 OneHotEncoder_d8241e2532b3,
 StringIndexer_fb478768f480,
 OneHotEncoder_a058bbea9c36,
 StringIndexer_7e8bfb78b9a0,
 OneHotEncoder_9ec09f4c665e,
 StringIndexer_d2c9c18549b8,
 VectorAssembler_bb3e7e75c61e]

In [ ]:
# Create a Pipeline.
pipeline = Pipeline(stages=stages)
pipelineModel = pipeline.fit(df_remove)
model = pipelineModel.transform(df_remove)

In [ ]:
model.take(1)

[Row(x=1, age=25.0, age_square=625.0, workclass='Private', fnlwgt=226802.0, education='11th', education_num=7.0, marital_status='Never-married', occupation='Machine-op-inspct', relationship='Own-child', race='Black', gender='Male', capital_gain=0.0, capital_loss=0.0, hours_week=40.0, native_country='United-States', label='<=50K', workclassIndex=0.0, workclassclassVec=SparseVector(8, {0: 1.0}), educationIndex=5.0, educationclassVec=SparseVector(15, {5: 1.0}), marital_statusIndex=1.0, marital_statusclassVec=SparseVector(6, {1: 1.0}), occupationIndex=6.0, occupationclassVec=SparseVector(14, {6: 1.0}), relationshipIndex=2.0, relationshipclassVec=SparseVector(5, {2: 1.0}), raceIndex=1.0, raceclassVec=SparseVector(4, {1: 1.0}), genderIndex=0.0, genderclassVec=SparseVector(1, {0: 1.0}), native_countryIndex=0.0, native_countryclassVec=SparseVector(40, {0: 1.0}), newlabel=0.0, features=SparseVector(99, {0: 1.0, 13: 1.0, 24: 1.0, 35: 1.0, 45: 1.0, 49: 1.0, 52: 1.0, 53: 1.0, 93: 25.0, 94: 226802.

## Step 4) Build the classifer: logisic

In [ ]:
from pyspark.ml.linalg import DenseVector
input_data = model.rdd.map(lambda x: (x["newlabel"], DenseVector(x["features"])))

In [ ]:
df_train = sqlContext.createDataFrame(input_data, ["label", "features"])

In [ ]:
df_train.show(2)

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|[1.0,0.0,0.0,0.0,...|
|  0.0|[1.0,0.0,0.0,0.0,...|
+-----+--------------------+
only showing top 2 rows



In [ ]:
# Create a train/test set
train_data, test_data = df_train.randomSplit([.8,.2],seed=1234)

In [ ]:
train_data.groupby('label').agg({'label': 'count'}).show()

+-----+------------+
|label|count(label)|
+-----+------------+
|  0.0|       29701|
|  1.0|        9345|
+-----+------------+



In [ ]:
test_data.groupby('label').agg({'label': 'count'}).show()

+-----+------------+
|label|count(label)|
+-----+------------+
|  0.0|        7453|
|  1.0|        2342|
+-----+------------+



In [ ]:
#Build the logisic regressor
from pyspark.ml.classification import LogisticRegression

# Initialize `lr`
lr = LogisticRegression(labelCol="label",
                        featuresCol="features",
                        maxIter=10,
                        regParam=0.3)

# Fit the data to the model
linearModel = lr.fit(train_data)

In [ ]:
# Print the coefficients and intercept for logistic regression
print("Coefficients: " + str(linearModel.coefficients))
print("Intercept: " + str(linearModel.intercept))

Coefficients: [-0.01214505924991046,-0.06823860934764871,0.02590973176806412,-0.15292515783828683,-0.044840458857517224,0.27970562888971845,0.24647580835764304,-0.4560424712738613,-0.1365223031970391,-0.03315220059099741,0.22625745712162823,0.3701547467909734,0.0037020958989733704,-0.22829460750030028,0.019811878548210746,-0.23459262516102183,-0.2607630592044288,0.5250358407234751,-0.23543784877427493,-0.19113553874878203,0.5107503253509824,-0.22817086177572463,-0.2610155931858554,0.411606905887486,-0.31119334767374346,-0.18014793011827138,-0.18777770284996675,-0.15923275299794762,-0.15174412850202196,0.24989942019871186,-0.03146648002530881,0.33886264346132794,-0.09526103834724899,0.06129512879112693,-0.25564705012497235,-0.16869242642823615,-0.15344218378465108,-0.06958149377296703,-0.2180335954061276,-0.2754137819908856,0.10549640739397056,0.11078385575328129,-0.22011813710286274,0.3434580549808888,-0.19388305268234202,-0.27958466874037746,-0.2138644021840045,0.4479481504436871,0.08

## Step 5) Train and evaluate the model

In [ ]:
# Make predictions on test data using the transform() method.
predictions = linearModel.transform(test_data)

In [ ]:
predictions.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [ ]:
selected = predictions.select("label", "prediction", "probability")
selected.show(20)

+-----+----------+--------------------+
|label|prediction|         probability|
+-----+----------+--------------------+
|  0.0|       0.0|[0.94897211752093...|
|  0.0|       0.0|[0.96181340180899...|
|  0.0|       0.0|[0.81544680161191...|
|  0.0|       0.0|[0.93838588013706...|
|  0.0|       0.0|[0.55062963693331...|
|  0.0|       1.0|[0.28223281796724...|
|  0.0|       1.0|[0.34214769708732...|
|  0.0|       0.0|[0.92861466273695...|
|  0.0|       1.0|[0.41427127276005...|
|  0.0|       1.0|[0.31821655911344...|
|  0.0|       0.0|[0.91805481465498...|
|  0.0|       0.0|[0.87671197102811...|
|  0.0|       0.0|[0.86521764677453...|
|  0.0|       0.0|[0.94658924875362...|
|  0.0|       0.0|[0.67979060749181...|
|  0.0|       0.0|[0.78793006516059...|
|  0.0|       0.0|[0.85662515535784...|
|  0.0|       0.0|[0.84317330097427...|
|  0.0|       0.0|[0.84126323762299...|
|  0.0|       0.0|[0.86178242748983...|
+-----+----------+--------------------+
only showing top 20 rows



In [ ]:
# Evaluate the model
cm = predictions.select("label", "prediction")
cm.groupby('label').agg({'label': 'count'}).show()

+-----+------------+
|label|count(label)|
+-----+------------+
|  0.0|        7453|
|  1.0|        2342|
+-----+------------+



In [ ]:
cm.groupby('prediction').agg({'prediction': 'count'}).show()

+----------+-----------------+
|prediction|count(prediction)|
+----------+-----------------+
|       0.0|             8768|
|       1.0|             1027|
+----------+-----------------+



In [ ]:
cm.filter(cm.label == cm.prediction).count() / cm.count()

0.8265441551812149

In [ ]:
#We need to look at the accuracy metric to see how well (or bad) the model performs.

def accuracy_m(model):
    predictions = model.transform(test_data)
    cm = predictions.select("label", "prediction")
    acc = cm.filter(cm.label == cm.prediction).count() / cm.count()
    print("Model accuracy: %.3f%%" % (acc * 100))

accuracy_m(model = linearModel)

Model accuracy: 82.654%


In [ ]:
### ROC metric
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Evaluate model
evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")
print(evaluator.evaluate(predictions))
print(evaluator.getMetricName())

0.8879099229638652
areaUnderROC


# Step 6) Tune the hyperparameter

In [ ]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

# Create ParamGrid for Cross Validation
paramGrid = (ParamGridBuilder()
             .addGrid(lr.regParam, [0.01, 0.5])
             .build())

In [ ]:
from time import *
start_time = time()

# Create 5-fold CrossValidator
cv = CrossValidator(estimator=lr,
                    estimatorParamMaps=paramGrid,
                    evaluator=evaluator, numFolds=5)

# Run cross validations
cvModel = cv.fit(train_data)
# likely take a fair amount of time
end_time = time()
elapsed_time = end_time - start_time
print("Time to train model: %.3f seconds" % elapsed_time)

Time to train model: 657.610 seconds


In [49]:
#accuracy of cv selected model
accuracy_m(model = cvModel)

Model accuracy: 84.941%


In [50]:
bestModel = cvModel.bestModel
bestModel.extractParamMap()

{Param(parent='LogisticRegression_962e51bd17ff', name='aggregationDepth', doc='suggested depth for treeAggregate (>= 2).'): 2,
 Param(parent='LogisticRegression_962e51bd17ff', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.0,
 Param(parent='LogisticRegression_962e51bd17ff', name='family', doc='The name of family which is a description of the label distribution to be used in the model. Supported options: auto, binomial, multinomial'): 'auto',
 Param(parent='LogisticRegression_962e51bd17ff', name='featuresCol', doc='features column name.'): 'features',
 Param(parent='LogisticRegression_962e51bd17ff', name='fitIntercept', doc='whether to fit an intercept term.'): True,
 Param(parent='LogisticRegression_962e51bd17ff', name='labelCol', doc='label column name.'): 'label',
 Param(parent='LogisticRegression_962e51bd17ff', name='maxBlockSizeInMB', doc='maximum memory in MB for s